In [107]:
from pathlib import Path

In [108]:
from collections import Counter

In [109]:
from itertools import combinations

In [110]:
PATH = Path("categories.txt")

In [111]:
def load_transactions(path: Path):
    tx = []
    with path.open("r", encoding = "utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            items = [x.strip() for x in line.split(";") if x.strip()]
            tx.append(tuple(sorted(items)))
    return tx

In [112]:
def count_support(transactions, candidates):
    counter = Counter()
    cand_sets = [set(c) for c in candidates]
    for t in transactions:
        s = set(t)
        for c, cs in zip(candidates, cand_sets):
            if cs.issubset(s):
                counter[c] += 1
    return counter

In [113]:
def get_L1(transactions, min_sup_abs):
    c1 = Counter(item for t in transactions for item in t)
    L1 = { (i,): cnt for i, cnt in c1.items() if cnt >= min_sup_abs}

    return dict(sorted(L1.items()))

In [114]:
def generate_candidates(prev_Lk):
    """prev_Lk: dict with keys = tuples of length k-1 (sorted)"""
    prev_itemsets = sorted(prev_Lk.keys())
    k = len(prev_itemsets[0]) + 1 if prev_itemsets else 1
    Ck = set()
    # join step
    for i in range(len(prev_itemsets)):
        for j in range(i+1, len(prev_itemsets)):
            a, b = prev_itemsets[i], prev_itemsets[j]
            if a[:-1] == b[:-1]:  # 
                cand = tuple(sorted(set(a) | set(b)))
                if len(cand) == k:
                    # prune step: all (k-1) subsets must exist in prev_Lk
                    ok = True
                    for sub in combinations(cand, k-1):
                        if sub not in prev_Lk:
                            ok = False
                            break
                    if ok:
                        Ck.add(cand)
            else:
                break
    return sorted(Ck)


In [115]:
def apriori(transactions, min_sup_rel = 0.01):
    N = len(transactions)
    min_sup_abs = int(min_sup_rel * N + 1e-9)
    
    L = []
    L1 = get_L1(transactions, min_sup_abs)
    if not L1:
        return [], min_sup_abs

    L.append(L1)

    k = 2
    
    while True:
        Ck = generate_candidates(L[-1])
        if not Ck:
            break
        counts = count_support(transactions, Ck)
        Lk = {c: cnt for c, cnt in counts.items() if cnt >= min_sup_abs }
        if not Lk:
            break
        L.append(dict(sorted(Lk.items())))
        k += 1
    return L, min_sup_abs    

In [116]:
def save_part1(L_all, out_path="part1.txt"):
    L1 = L_all[0]
    with open(out_path, "w", encoding="utf-8") as f:
        for (item,), cnt in sorted(L1.items(), key=lambda x: (-x[1], x[0][0])):
            f.write(f"{cnt}:{item}\n")            

In [117]:
def save_part2(L1_all, out_path="part2.txt"):
    with open(out_path, "w", encoding="utf=8") as f:
        for Lk in L_all:
            for items, cnt in sorted(Lk.items(), key=lambda x: (-x[1], x[0])):
                line_items = ";".join(items)
                f.write(f"{cnt}:{line_items}\n")

In [118]:
transactions = load_transactions(PATH)

In [119]:
N = len(transactions)

In [120]:
N

77185

In [121]:
L_all, min_sup_abs = apriori(transactions, min_sup_rel = 0.01)

In [122]:
L_all

[{('Active Life',): 3103,
  ('American (New)',): 1593,
  ('American (Traditional)',): 2416,
  ('Arts & Entertainment',): 2271,
  ('Auto Repair',): 1716,
  ('Automotive',): 4208,
  ('Bakeries',): 1115,
  ('Bars',): 4328,
  ('Beauty & Spas',): 6583,
  ('Breakfast & Brunch',): 1369,
  ('Burgers',): 1774,
  ('Cafes',): 1002,
  ('Chinese',): 1629,
  ('Coffee & Tea',): 2199,
  ('Dentists',): 1195,
  ('Doctors',): 1694,
  ('Event Planning & Services',): 2975,
  ('Fashion',): 3078,
  ('Fast Food',): 2851,
  ('Financial Services',): 875,
  ('Fitness & Instruction',): 1442,
  ('Food',): 9250,
  ('General Dentistry',): 823,
  ('Grocery',): 1424,
  ('Hair Salons',): 2091,
  ('Health & Medical',): 5121,
  ('Home & Garden',): 1586,
  ('Home Services',): 4785,
  ('Hotels',): 1431,
  ('Hotels & Travel',): 2495,
  ('Ice Cream & Frozen Yogurt',): 1018,
  ('Italian',): 1848,
  ('Japanese',): 848,
  ('Local Services',): 3468,
  ('Mexican',): 2515,
  ('Nail Salons',): 1667,
  ('Nightlife',): 5088,
  ('Pet 

In [123]:
min_sup_abs

771

In [124]:
sum(len(Lk) for Lk in L_all), [len(Lk) for Lk in L_all], min_sup_abs

(103, [50, 45, 8], 771)

In [125]:
list(L_all[0].items())[:5]

[(('Active Life',), 3103),
 (('American (New)',), 1593),
 (('American (Traditional)',), 2416),
 (('Arts & Entertainment',), 2271),
 (('Auto Repair',), 1716)]

In [126]:
for k, Lk in enumerate(L_all, start=1):
    print(f"k={k} -> {len(Lk)} itemsets")

k=1 -> 50 itemsets
k=2 -> 45 itemsets
k=3 -> 8 itemsets


In [127]:
save_part1(L_all, "part1.txt")

In [128]:
save_part2(L_all, "part2.txt")